## Problem statement
Day 6's problem consists of being given column-wise operations instructions and finding the sum of these instructions.

Let's load the inputs.

In [1]:
import numpy as np
with open('sample_input.txt', 'r') as f:
    lines1 = f.read().splitlines()


with open('puzzle_input.txt', 'r') as f:
    lines2 = f.read().splitlines()

def data_cleaner(lines: list[str]) -> np.ndarray:
    #Split the lines on whitespaces
    lines = [line.split(' ') for line in lines]
    #Remove empty characters
    lines = [[c for c in line if c != ''] for line in lines]
    
    return np.array(lines)


lines1 = data_cleaner(lines1)
lines2 = data_cleaner(lines2)

print(lines1)

[['123' '328' '51' '64']
 ['45' '64' '387' '23']
 ['6' '98' '215' '314']
 ['*' '+' '*' '+']]


### Performing the operations based on the column instructions
Now all that's left is to read the operation at the bottom of the columns and add the column-wise numbers obtained from following the instructions and sum them.

In [2]:
def operations_sum(lines: list[str]) -> np.ndarray:
    rows = len(lines)
    cols = len(lines[0])
    
    total = []
    for i in range(cols):
        symbol = lines[-1][i]
        col = lines[:rows-1, i]
    
        if symbol == '+':
            total.append(str(np.sum(col.astype(int))))
            
        else:
            total.append(str(np.prod(col.astype(int))))

    return np.array(total)

sample_sum = operations_sum(lines1)
puzzle_sum = operations_sum(lines2)
print(f"Sample input solution is: {np.sum(sample_sum.astype(int))}")
print(f"Puzzle input solution is: {np.sum(puzzle_sum.astype(int))}")

Sample input solution is: 4277556
Puzzle input solution is: 6417439773370


## EXTRA: Solution idea I had in my mind originally
Initially, I thought that the sum would probably cause an integer overflow due to the large number of operations and the potential for the numbers to be really really big. However, it **turns out python integers have arbitrary precision** so I did not need to do this!

Anyways, looking below you'll see what I had in mind. It's based on the way we add numbers on paper, where if when we add two digits and their sum > 9, we add one to the left and keep the rightmost digit of the sum.

I used a recursive dictionary to allow us store the digits of large length numbers. Since I stored the operation numbers to be added as strings, we iterate through them from right to left and either:
1. Update the dictionary entry and if the sum is greater than 9 we recur
2. Create a new one if a previous one didn't exist

In [3]:
def add_mapping(item: str, indx: int, d: dict) -> None:
    indx = str(indx)
    if indx in d:
        d[indx] = str(int(d[indx]) + int(item))
        if len(d[indx]) > 1:
            item = d[indx][0]
            d[indx] = d[indx][1]
            indx = str(int(indx) + 1)
            add_mapping(item, indx, d)
    else:
        d[indx] = item
        
d = {}   
for l in puzzle_sum:
    for index, item in enumerate(l[::-1]):
        add_mapping(item, index, d)

values = list(d.values())[::-1]

ans = ""
for c in values:
    ans += c

print(ans)

6417439773370


## Part 2
Well now it turns out that these weirdos read numbers in a digit right-to-left column fashion. For example:
```
64 
23 
314
+  
```

Becomes {0}{0}4 (rightmost col) + 431 (middle col) + 623 (leftmost col) = 1058. 

Unfortunately, the way we read the data is not compatible with this sort of math operations as we lose crucial information. Therefore, we must reload the data and adapt our code to read numbers this way too.

In [5]:
#Reading file without removing the whitespace necessary for solving

with open('sample_input.txt', 'r') as f:
    lines1 = f.read().splitlines()
    
with open('puzzle_input.txt', 'r') as f:
    lines2 = f.read().splitlines()

def operations_sum2(lines: list[str]) -> None:
    #Exclude operator row
    rows = len(lines) - 1
    cols = len(lines[0])
    
    lst = [[] for _ in range(cols)]
    
    #Make a list of numbers as if you were reading them vertically
    for j in range(cols):
        for i in range(rows):
            lst[j].append(lines[i][j])
    
    #Group them by creating lists that are separated by ' '+ columns 
    #The columns are 3 whitespaces long for the sample and 4 for the input, hence why it's dynamic
    temp = []
    final = []
    check = [' ' for _ in range(rows)]
    for line in lst:
        if line == check:
            final.append(temp)
            temp = []
    
        #Concatenate the numbers, including any leading or trailing ' '
        else:
            x = ""
            for c in line:
                x += c
            temp.append(x)
            
    final.append(temp)
    
    #Clean up the operations instructions
    instructions = lines[-1].split(' ')
    instructions = [c for c in instructions if c != '']
    
    #Perform the operations
    total = 0
    for i in range(len(final)):
        symbol = instructions[i]
        if symbol == '+':
            temp = 0
            for n in final[i]:
                temp += int(n)
            total += temp
            
        else:
            temp = 1
            for n in final[i]:
                temp *= int(n)
            total += temp
    
    print(total)
    
operations_sum2(lines1)
operations_sum2(lines2)

3263827
11044319475191
